在上一篇中，我们手动完成了 RAG 的工作流程，但是实际使用当中，我们需要它自动进行。在这里，我们将修改代码，让这个工作流程自动化运行。

首先我们列出所有的预设和函数声明。

### Global Configurations

In [1]:
import os
import re
import gc
import ast
import time
import base64
import ollama
import numpy as np
from io import BytesIO
from pdf2image import convert_from_path
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed


# ====================== Model Configuration ======================
OCR_MODEL = "glm-ocr"
OCR_DPI = 100
OCR_PROMPT = "Text recognition. Accurately extract all text, LaTeX formulas, and symbols from the image. Output only the original content."
OLLAMA_HOST = "http://localhost:11434"

LLM_MODEL = "gpt-oss:20b"

EMBEDDING_MODEL = "embeddinggemma"

MAX_WORKERS = 8  # The Threads of Embedding. Suit your CPU. Higher is better.

# ====================== Global Variables ======================
FILES_PATH = "./pdfs"

full_embeddings = {}
categories_list = {}
full_chunks = {}
categories_list["General Document"] = []

### OCR

In [2]:
# ====================== Image to Base64 ======================
def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG", quality=80, optimize=True)
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

# ====================== OCR single page ======================
def ocr_single_page(page_img):
    img_base64 = image_to_base64(page_img)
    response = ollama.chat(
        model=OCR_MODEL,
        messages=[{'role': 'user', 'content': OCR_PROMPT, 'images': [img_base64]}],
    )
    return response["message"]["content"].strip()

def ocr_single_pdf(pdf_path):
    full_text = ""
    try:
        # Get PDF file name
        pdf_file_name = os.path.basename(pdf_path)
        
        print(f"\n🚀 GLM-OCR processing: {pdf_file_name}")
        
        # Convert PDF to images
        pages = convert_from_path(
            pdf_path, dpi=OCR_DPI, thread_count=1, use_pdftocairo=True, grayscale=True
        )
        total_pages = len(pages)
    
        # Iterate through each page to process OCR
        for idx, page_img in enumerate(pages):
            page_text = ""
            # Retry mechanism (max 2 attempts)
            for retry in range(2):
                try:
                    # OCR 
                    page_text = ocr_single_page(page_img)
                    break  # Exit retry if successful
                except TimeoutError:
                    print(f"   ⚠️ Page {idx+1} OCR timed out, retrying {retry+1}...")
                    time.sleep(2)
                except Exception as e:
                    print(f"   ⚠️ Page {idx+1} OCR failed: {e}")
                    time.sleep(2)
            
            # Fill placeholder text when OCR fails
            if not page_text:
                page_text = f"[Recognition failed for Page {idx+1}]"
            
            # Concatenate text
            full_text += page_text + "\n\n"
            
    
        # Clean up redundant whitespace characters
        full_text = re.sub(r'\s+', ' ', full_text).strip()
        print(f"✅ OCR completed! Text length: {len(full_text)} characters")

    except Exception as e:
        print(f"❌ OCR failed: {str(e)}")
        full_text = ""

    clean_text = full_text.replace('"', '').replace("'", "").replace("[", "").replace("]", "").replace("{", "").replace("}", "")

    return clean_text

### List all files in a directory

In [3]:
def list_all_files_in_dir(path):
    extensions=['.pdf']
    
    if not os.path.exists(path):
        print(f"错误：路径 '{path}' 不存在")
        return []
    
    extensions_lower = [ext.lower() for ext in extensions]
    file_list = []
    
    for item in os.listdir(path):
        item_full_path = os.path.join(path, item)
        if os.path.isfile(item_full_path):
            file_ext = os.path.splitext(item)[1].lower()
            if file_ext in extensions_lower:
                file_list.append(item_full_path)
    
    return file_list

### Classify Categories

In [4]:
def classify_categories(file_path, full_text):
    prompt = f"""
Based on the following content, output only Numpy Dict with no other content:
{{"categories":["..."]}}
    
Content:
{full_text[:1000]}
"""
    
    result = ollama.chat(model=LLM_MODEL, messages=[{"role":"user","content":prompt}])
    cat_tags = ast.literal_eval(result["message"]["content"])

    categories_list["General Document"] = [file_path]
        
    for category in cat_tags["categories"]:
        if category not in categories_list:
            categories_list[category] = []
        categories_list[category].append(file_path)
    return categories_list

### Split Text to Chunks

In [5]:
def split_text_chunks(text, chunk_size=500, chunk_overlap=50):
    chunks = []
    start = 0
    text_length = len(text)
    # Priority separators (semantic order: paragraph > sentence > punctuation)
    separators = ["\n\n", "\n", "\'", "。", "！", "？", "；", "，", "、", "：", ".", "!", "?", ";", ","]

    while start < text_length:
        end = start + chunk_size
        # Add remaining text as last chunk if end exceeds text length
        if end >= text_length:
            chunks.append(text[start:].strip())
            break
        
        temp_chunk = text[start:end]
        split_pos = -1
        # Find last valid separator (after overlap threshold)
        for sep in separators:
            pos = temp_chunk.rfind(sep)
            if pos != -1 and pos > (chunk_size - chunk_overlap):
                split_pos = pos
                break
        
        # Split at natural separator if found
        if split_pos != -1:
            chunk_end = start + split_pos + 1
            chunks.append(text[start:chunk_end].strip())
            start = chunk_end - chunk_overlap
        # Fallback: split at chunk size with overlap
        else:
            chunks.append(temp_chunk.strip())
            start = end - chunk_overlap

    # Filter out empty chunks
    return [c for c in chunks if c]

### Embedding

In [6]:
def embed_chunk(chunk):
    try:
        #  embedding chunk
        res = ollama.embed(model=EMBEDDING_MODEL, input=chunk)
        # Convert embedding to float32 numpy array for efficiency
        return np.array(res["embeddings"], dtype=np.float32)
    except:
        # Return empty array if embedding generation fails
        return np.array([])

def parallel_get_embeddings(chunks):
    all_emb = []
    # Use process pool for parallel embedding (faster for CPU/GPU-bound tasks)
    with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Submit embedding tasks for all chunks
        futs = [executor.submit(embed_chunk, b) for b in chunks]
        # Collect valid embedding results
        for f in futs:
            be = f.result()
            if len(be) > 0:
                all_emb.append(be[0])
                
    if all_emb:
        final = np.vstack(all_emb)
        print(f"✅ Finished Embedding")
        return final
    # Return empty array if no valid embeddings
    return np.array([])

### Some function for usages

In [7]:
def merge_two_dict_lists(dict1, dict2):
    merged_dict = {}
    for key, value in dict1.items():
        merged_dict[key] = value.copy() if isinstance(value, list) else [value]
    
    for key, value in dict2.items():
        list_value = value.copy() if isinstance(value, list) else [value]
        
        if key in merged_dict:
            merged_dict[key] += list_value
        else:
            merged_dict[key] = list_value
    
    return merged_dict
    
def cosine_similarity(vec1, vec2):
    n1 = np.linalg.norm(vec1)
    n2 = np.linalg.norm(vec2)
    
    if n1 == 0 or n2 == 0:
        return 0.0
    
    return np.dot(vec1/n1, vec2/n2)

### Find best Category

In [8]:
def find_best_category(question):
    max_similarity = -1.0  # Track highest similarity score
    best_category = ""     # Track category with highest similarity
    question_embedding = ollama.embed(model=EMBEDDING_MODEL, input=question)["embeddings"][0]
    # Iterate through all categories to find most similar one
    for category in categories_list:
        # Generate embedding for current category and calculate similarity to question
        similarity = cosine_similarity(question_embedding, ollama.embed(model=EMBEDDING_MODEL, input=category)["embeddings"][0])
        
        # Print category (left-aligned) and similarity (12 decimal places)
        print(f"{category:<{20}} {similarity:.{12}f}")
        
        # Update top category if current similarity is higher
        if similarity > max_similarity:
            max_similarity = similarity
            best_category = category
    
    return best_category

### Get relevant chunks

In [9]:
def get_relevant_chunks(question):
    global full_embeddings, full_chunks
    results = []
    question_embedding = ollama.embed(model=EMBEDDING_MODEL, input=question)["embeddings"][0]
    for file in full_embeddings:
        index = 0
        files = list_all_files_in_dir(FILES_PATH)
        for file in files:
            for idx, embedding in enumerate(full_embeddings[file]):
                similarity = cosine_similarity(question_embedding, embedding)
                results.append((idx, similarity, file, full_chunks[file][idx]))
    
    seen = set()
    unique_results = []
    for item in results:
        if item not in seen:
            seen.add(item)
            unique_results.append(item)
    
    unique_results.sort(key=lambda x: x[1], reverse=True)
    return unique_results[:3]

## Work Flow
### Process: Prepare embeddings, categories and anything we need

In [10]:
def get_all_pdfs_embeddings(pdf_path):
    global categories_list, full_embeddings, full_chunks
    
    files = list_all_files_in_dir(pdf_path)
    for file in files:
        full_text = ocr_single_pdf(file)
        categories_list = merge_two_dict_lists(categories_list, classify_categories(file, full_text))
        chunks = split_text_chunks(full_text)
        full_chunks[file] = chunks
        embeddings = parallel_get_embeddings(chunks)
        full_embeddings[file] = embeddings
        
get_all_pdfs_embeddings(FILES_PATH)


🚀 GLM-OCR processing: 摄影术语：log、动态范围和色彩取样.pdf
✅ OCR completed! Text length: 5996 characters
✅ Finished Embedding

🚀 GLM-OCR processing: What is the differences between gcc and Clang.pdf
✅ OCR completed! Text length: 15804 characters
✅ Finished Embedding

🚀 GLM-OCR processing: How to Adjust NVIDIA GPU Frequency, Volte and Other Settings on Linux and Overlocking.pdf
✅ OCR completed! Text length: 14016 characters
✅ Finished Embedding

🚀 GLM-OCR processing: Use gcc to learn C compilation process.pdf
   ⚠️ Page 9 OCR failed: an error was encountered while running the model: GGML_ASSERT(a->ne[2] * 4 == b->ne[0]) failed (status code: 500)
   ⚠️ Page 9 OCR failed: an error was encountered while running the model: GGML_ASSERT(a->ne[2] * 4 == b->ne[0]) failed (status code: 500)
✅ OCR completed! Text length: 13109 characters
✅ Finished Embedding

🚀 GLM-OCR processing: Maximum Likelihood Estimation with Code.pdf
✅ OCR completed! Text length: 18227 characters
✅ Finished Embedding

🚀 GLM-OCR process

### Ask questions

In [11]:
def generate_answer(question):
    relevant_chunks = get_relevant_chunks(question)
    # concentrate chunks together 
    ctx = "\n\n".join([f"{item[1][2]}" for item in enumerate(relevant_chunks)])
    prompt = f"""Please answer truthfully based on the context, prioritize extracting specific numbers and details, say 'I don't know' if you don't know, and do not fabricate content. If you know the answer, please in ask language
Context:
{ctx}
Question: {question}"""
    try:
        resp = ollama.chat(model=LLM_MODEL, messages=[{"role":"user","content":prompt}])
        return resp["message"]["content"]
    except:
        return "Failed to generate answer"
        
question="如何在Ubuntu系统上手动控制风扇转速?"
answer = generate_answer(question)
print(answer)

**在 Ubuntu 上手动控制风扇转速（常见方法）**

> 下面的步骤兼顾一般桌面主板风扇（需要 `lm‑sensors` 与 `fancontrol`）以及 NVIDIA GPU 的风扇。  
> 如果你只想调 GPU 风扇，后面“GPU 风扇”部分即可；若是整个机箱风扇，请按“主板风扇”部分操作。

---

## 1. 主板/机箱风扇（PWM 方式）

| 步骤 | 命令 / 操作 | 说明 |
|------|-------------|------|
| 1 | `sudo apt update && sudo apt install lm-sensors fancontrol` | 安装感应与控制工具 |
| 2 | `sudo sensors-detect` 并按提示 `yes` | 识别硬件传感器与 PWM 芯片 |
| 3 | `sudo pwmconfig` | 自动生成 `/etc/fancontrol` 并测试 PWM（按提示选择 0–255） |
| 4 | 打开 `/etc/fancontrol`，编辑 `MINSTART`, `MINSTOP`, `MAXSTART`, `MAXSTOP` 等字段 | 这些值决定风扇最低/最高转速与温度阈值 |
| 5 | 启动服务：`sudo systemctl enable --now fancontrol` | 让 `fancontrol` 自动在启动时运行 |
| 6 | 检查：`cat /sys/class/hwmon/hwmon*/pwm1` 或 `pwm2` | 查看当前 PWM 数值（0–255） |
| 7 | 手动修改：`sudo sh -c 'echo 180 > /sys/class/hwmon/hwmon*/pwm1'` | 180 对应约 70% 风扇转速（取决于芯片） |

> **常见数字**  
> - 0 → 关闭或最低转速  
> - 255 → 最大转速  
> - 120–180 → 约 40–70%（根据主板手册校准）  

---

## 2. NVIDIA GPU 风扇（`nvidia-settings`）

> 需要 NVIDIA 驱动已正确安装且 `nvidia-settings` 可执行。

| 步骤 | 命令 | 说明 |
|------|--